In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import urllib
from selenium import webdriver
from time import sleep
import re
import os

In [57]:
#CHANGE THIS WITH THE RIGHT PATH!
chromedriver = "/Users/darrenchia/Desktop/DSI_Projects/project-4/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

# Visit our relevant page.
url = "https://www.mycareersfuture.sg/search?search=Data&postingCompany=Direct&sortBy=new_posting_date&page={}"
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver.get(url.format(0))
assert "MyCareersFuture" in driver.title

# loop through to 100 pages, each page contains 20 job postings
# for num in range(100):
#     driver.get(url.format(num))

# Set sleep interval
t = np.random.randint(7,9) 
sleep(t)
# Grab the page source.
html = driver.page_source
# Beautiful Soup it!
soup = BeautifulSoup(html, 'lxml')



In [3]:
# driver.title

'Search results | MyCareersFuture'

In [58]:
# Look for all weblinks and location of each job posting
urls = []
location = []

for page in range(0,101):
    driver.get(url.format(page))
    # Set sleep interval
    t = np.random.randint(7,9) 
    sleep(t)
    # Grab the page source.
    html = driver.page_source
    # Beautiful Soup it!
    soup = BeautifulSoup(html, 'lxml')
    for entry in soup.find_all('a', {'class':'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'}):
        partial_url = entry.get('href')
        full_url = 'https://www.mycareersfuture.sg' + partial_url
        urls.append(full_url)
        try:
            location.append(entry.find('p', {'name':'location'}).text)
        except:
            location.append(None)
    print('-'*5, 'pages scraped:', page+1, '-'*5)

----- pages scraped: 1 -----
----- pages scraped: 2 -----
----- pages scraped: 3 -----
----- pages scraped: 4 -----
----- pages scraped: 5 -----
----- pages scraped: 6 -----
----- pages scraped: 7 -----
----- pages scraped: 8 -----
----- pages scraped: 9 -----
----- pages scraped: 10 -----
----- pages scraped: 11 -----
----- pages scraped: 12 -----
----- pages scraped: 13 -----
----- pages scraped: 14 -----
----- pages scraped: 15 -----
----- pages scraped: 16 -----
----- pages scraped: 17 -----
----- pages scraped: 18 -----
----- pages scraped: 19 -----
----- pages scraped: 20 -----
----- pages scraped: 21 -----
----- pages scraped: 22 -----
----- pages scraped: 23 -----
----- pages scraped: 24 -----
----- pages scraped: 25 -----
----- pages scraped: 26 -----
----- pages scraped: 27 -----
----- pages scraped: 28 -----
----- pages scraped: 29 -----
----- pages scraped: 30 -----
----- pages scraped: 31 -----
----- pages scraped: 32 -----
----- pages scraped: 33 -----
----- pages scraped

In [59]:
len(urls)

1900

In [60]:
urls[:5]

['https://www.mycareersfuture.sg/job/data-engineer-moneysmart-singapore-d56b1a5839b83d1bf6486c4beb22ed91',
 'https://www.mycareersfuture.sg/job/data-scientist-portcast-3571a3c590a2af44ecf32d28aa1dfe07',
 'https://www.mycareersfuture.sg/job/data-center-engineering-project-engineer-apac-amazon-asia-pacific-resources-de31764deee5ad19b1c31315dad0c575',
 'https://www.mycareersfuture.sg/job/lead-data-center-engineer-johnson-johnson-c0791839b727bc55b28d04045e79a617',
 'https://www.mycareersfuture.sg/job/data-scientist-intern-jabil-circuit-332041884350de0452d66a9ace0cf95e']

In [61]:
len(location)

1900

In [62]:
location[:5]

['South', 'South', 'Central', 'Central', 'East']

In [65]:
loc_url = pd.DataFrame()

In [66]:
loc_url['urls']=urls

In [67]:
loc_url['location']=location

In [69]:
loc_url.shape

(1900, 2)

In [70]:
loc_url.head()

,urls,location
0,https://www.mycareersfuture.sg/job/data-engine...,South
1,https://www.mycareersfuture.sg/job/data-scient...,South
2,https://www.mycareersfuture.sg/job/data-center...,Central
3,https://www.mycareersfuture.sg/job/lead-data-c...,Central
4,https://www.mycareersfuture.sg/job/data-scient...,East


In [71]:
loc_url.to_csv('loc_url.csv')

In [72]:
# create empty dataframe to input info later
jobs_df = pd.DataFrame(columns=['coy_name','employment_type','job_title', 'seniority', 
                             'salary', 'salary_type','job_description','requirement'])

In [73]:
# Running through all the weblinks to scrape the required data
for i, v in enumerate(urls):

    driver.get(v)
    # Set sleep interval
    t = np.random.randint(7,9) 
    sleep(t)
    # Grab the page source.
    html = driver.page_source
    # Beautiful Soup it!
    soup = BeautifulSoup(html, 'lxml')
        
    # company name
    try:
        coy_name = soup.find('p', {'name': 'company'}).text
    except:
        coy_name = np.nan
        
   # employment type
    try:
        employment_type = soup.find('p', {'id': 'employment_type'}).text
    except:
        employment_type = np.nan
        
    # job title
    try:
        job_title = soup.find('h1', {'id': 'job_title'}).text
    except:
        job_title = np.nan
        
    # seniority
    try:
        seniority = soup.find('p', {'id': 'seniority'}).text
    except:
        seniority = np.nan
        
    # salary
    try:
        salary = soup.find('div', {'class': 'lh-solid'}).text
    except:
        salary = np.nan   
    
    # salary type
    try:
        salary_type = soup.find('span', {'class': 'salary_type dib f5 fw4 black-60 pr1 i pb'}).text
    except:                                     
        salary_type = np.nan     
        
    # job_description
    try:
        job_description = soup.find('div', {'id': 'description-content'}).text
    except:
        job_description = np.nan
        
    # requirement
    try:
        requirement = soup.find('div', {'id': 'requirements-content'}).text
    except:
        requirement = np.nan 
    
    # print no. of job postings scraped per 10
    if i%10==0:
        print('-'*5, 'jobs scraped:', i, '-'*5)

    # save to csv after every 100 job postings scraped
    if i%100==0:
        jobs_df.to_csv('./jobs_data.csv')        
    
    # Add values to each row of the dataframe
    jobs_df.loc[len(jobs_df)] = [coy_name, employment_type, job_title, seniority, 
                             salary, salary_type, job_description, requirement]
        

----- jobs scraped: 0 -----
----- jobs scraped: 10 -----
----- jobs scraped: 20 -----
----- jobs scraped: 30 -----
----- jobs scraped: 40 -----
----- jobs scraped: 50 -----
----- jobs scraped: 60 -----
----- jobs scraped: 70 -----
----- jobs scraped: 80 -----
----- jobs scraped: 90 -----
----- jobs scraped: 100 -----
----- jobs scraped: 110 -----
----- jobs scraped: 120 -----
----- jobs scraped: 130 -----
----- jobs scraped: 140 -----
----- jobs scraped: 150 -----
----- jobs scraped: 160 -----
----- jobs scraped: 170 -----
----- jobs scraped: 180 -----
----- jobs scraped: 190 -----
----- jobs scraped: 200 -----
----- jobs scraped: 210 -----
----- jobs scraped: 220 -----
----- jobs scraped: 230 -----
----- jobs scraped: 240 -----
----- jobs scraped: 250 -----
----- jobs scraped: 260 -----
----- jobs scraped: 270 -----
----- jobs scraped: 280 -----
----- jobs scraped: 290 -----
----- jobs scraped: 300 -----
----- jobs scraped: 310 -----
----- jobs scraped: 320 -----
----- jobs scraped: 3

In [74]:
jobs_df.shape

(1900, 8)

In [76]:
# add urls and location to dataframe
jobs_df['location'] = location
jobs_df['urls'] = urls

In [77]:
jobs_df.shape

(1900, 10)

In [78]:
jobs_df.head(3)

,coy_name,employment_type,job_title,seniority,salary,salary_type,job_description,requirement,location,urls
0,MONEYSMART SINGAPORE PTE. LTD.,Full Time,Data Engineer,Executive,"$5,000to$7,000",Monthly,Mission As part of becoming one of the members...,Competencies Degree in Computer Science or an...,South,https://www.mycareersfuture.sg/job/data-engine...
1,PORTCAST PTE. LTD.,Full Time,Data Scientist,Middle Management,"$2,200to$6,000",Monthly,"In this role, you will be responsible for anal...",● Comfortable working with large amounts of d...,South,https://www.mycareersfuture.sg/job/data-scient...
2,AMAZON ASIA-PACIFIC RESOURCES PRIVATE LIMITED,Full Time,Data Center Engineering Project Engineer APAC,Professional,"$9,000to$12,000",Monthly,The Data Center Global Services (DCGS) team is...,Basic Qualifications - Minimum 5 years of mis...,Central,https://www.mycareersfuture.sg/job/data-center...


In [79]:
# update to csv
jobs_df.to_csv('jobs_data.csv') 

In [ ]:
# to resume from last url scraped 
# urls[350:]
# t = np.random.randint(4,6) 

## first attempt

In [12]:
# create empty dataframe to input info later
mycf = pd.DataFrame(columns=['coy_name','employment_type','job_title', 'seniority', 
                             'salary', 'salary_type','job_description','requirement'])

# save to csv 
# mcf = pd.DataFrame().to_csv('mcf.csv')

In [13]:
# Running through all the weblinks to scrape the required data
for i, v in enumerate(urls):

    driver.get(v)
    # Set sleep interval
    t = np.random.randint(7,9) 
    sleep(t)
    # Grab the page source.
    html = driver.page_source
    # Beautiful Soup it!
    soup = BeautifulSoup(html, 'lxml')

# Variables:
# 'coy_name','employment_type','job_title', 'seniority', 
# 'salary', 'salary_type','job_description','requirements'
        
    # company name
    try:
        coy_name = soup.find('p', {'name': 'company'}).text
    except:
        coy_name = np.nan
        
   # employment type
    try:
        employment_type = soup.find('p', {'id': 'employment_type'}).text
    except:
        employment_type = np.nan
        
    # job title
    try:
        job_title = soup.find('h1', {'id': 'job_title'}).text
    except:
        job_title = np.nan
        
    # seniority
    try:
        seniority = soup.find('p', {'id': 'seniority'}).text
    except:
        seniority = np.nan
        
    # salary
    try:
        salary = soup.find('div', {'class': 'lh-solid'}).text
    except:
        salary = np.nan   
    
    # salary type
    try:
        salary_type = soup.find('span', {'class': 'salary_type dib f5 fw4 black-60 pr1 i pb'}).text
    except:                                     
        salary_type = np.nan     
        
    # job_description
    try:
        job_description = soup.find('div', {'id': 'description-content'}).text
    except:
        job_description = np.nan
        
    # requirement
    try:
        requirement = soup.find('div', {'id': 'requirements-content'}).text
    except:
        requirement = np.nan 
    
    # print no. of job postings scraped per 10
    if i%10==0:
        print('-'*5, 'jobs scraped:', i, '-'*5)

    # save to csv after every 100 job postings scraped
    if i%100==0:
        mycf.to_csv('./jobs_data.csv')        
    
    # Add values to each row of the dataframe
    mycf.loc[len(mycf)] = [coy_name, employment_type, job_title, seniority, 
                             salary, salary_type, job_description, requirement]
        

In [21]:
i = 2
print('-'*5, 'jobs scraped:', i, '-'*5)

----- jobs scraped: 2 -----


In [17]:
mycf

,coy_name,employment_type,job_title,seniority,salary,salary_type,job_description,requirement
0,KELLY SERVICES (SINGAPORE) PTE. LTD.,Contract,Systems Analyst (Agile),Professional,"$9,000to$13,500",Monthly,Job Description To build scalable and reliab...,"Job Requirements Degree in Computer Science, ..."
1,KELLY SERVICES (SINGAPORE) PTE. LTD.,Contract,Systems Analyst (Agile),Professional,"$6,000to$9,000",Monthly,Job Description To build scalable and reliab...,"Job Requirements Degree in Computer Science, ..."
2,GOOGLE ASIA PACIFIC PTE. LTD.,Full Time,"Data Science Lead, Large Customer Sales - Sing...",Executive,"$7,700to$15,400",Monthly,Company overview: Google is not a conventional...,Minimum qualifications: - Bachelor’s degree in...
3,TRAKOMATIC PTE. LTD.,Full Time,Data analyst,Professional,"$3,500to$4,500",Monthly,The Retail Data Analyst will lead and perform ...,"Requirements: A bachelor’s degree in math,..."
4,"VARIAN MEDICAL SYSTEMS PACIFIC, INC.",Permanent,Business Data Analyst,Fresh/entry level,NaN,,Position Summary & Core Responsibilities: Data...,Minimum Required Skills and Knowledge: Profic...
5,INTELLECT MINDS PTE. LTD.,Full Time,Data Engineer,Executive,"$5,000to$7,000",Monthly,Company Overview Intellect Minds is a Singapor...,Qualifications • Advanced working SQL knowledg...
6,KIMBERLEY CONSULTING PTE. LTD.,Permanent,Data Analyst,Executive,NaN,,Large commodities company Marina Bay Respo...,Requirements: Minimum Bachelor’s Degree in Ma...
7,GUMI ASIA PTE. LTD.,Full Time,Data Analyst,Junior Executive,"$2,800to$3,800",Monthly,Perform evaluations and problem investigation...,Minimum Diploma/ Degree in Statistics/Compute...
8,DBS BANK LTD.,Full Time,"AVP, Data Scientist, Business Analytics, Consu...",Senior Management,"$6,500to$11,700",Monthly,Business Function As the leading bank in Asia...,"Masters or equivalent degree in Statistics, A..."
9,NIOMETRICS (PTE.) LTD.,Permanent,High-Performance Data Engineer,Professional,"$5,500to$11,000",Monthly,WHAT WE DO We invite you to be part of our amb...,WHAT WE VALUE Bachelor’s or Higher Degree in ...


In [44]:
mycf['urls']=urls

In [45]:
mycf.head()

,coy_name,employment_type,job_title,seniority,salary,salary_type,job_description,requirement,urls
0,KELLY SERVICES (SINGAPORE) PTE. LTD.,Contract,Systems Analyst (Agile),Professional,"$9,000to$13,500",Monthly,Job Description To build scalable and reliab...,"Job Requirements Degree in Computer Science, ...",https://www.mycareersfuture.sg/job/systems-ana...
1,KELLY SERVICES (SINGAPORE) PTE. LTD.,Contract,Systems Analyst (Agile),Professional,"$6,000to$9,000",Monthly,Job Description To build scalable and reliab...,"Job Requirements Degree in Computer Science, ...",https://www.mycareersfuture.sg/job/systems-ana...
2,GOOGLE ASIA PACIFIC PTE. LTD.,Full Time,"Data Science Lead, Large Customer Sales - Sing...",Executive,"$7,700to$15,400",Monthly,Company overview: Google is not a conventional...,Minimum qualifications: - Bachelor’s degree in...,https://www.mycareersfuture.sg/job/data-scienc...
3,TRAKOMATIC PTE. LTD.,Full Time,Data analyst,Professional,"$3,500to$4,500",Monthly,The Retail Data Analyst will lead and perform ...,"Requirements: A bachelor’s degree in math,...",https://www.mycareersfuture.sg/job/data-analys...
4,"VARIAN MEDICAL SYSTEMS PACIFIC, INC.",Permanent,Business Data Analyst,Fresh/entry level,NaN,,Position Summary & Core Responsibilities: Data...,Minimum Required Skills and Knowledge: Profic...,https://www.mycareersfuture.sg/job/business-da...


In [40]:
mycf.salary.describe()

count                 37
unique                33
top       $3,500to$4,500
freq                   2
Name: salary, dtype: object

In [24]:
mycf.salary.dtype

dtype('O')

In [37]:
mycf.salary[:5]

0    $9,000to$13,500
1     $6,000to$9,000
2    $7,700to$15,400
3     $3,500to$4,500
4                NaN
Name: salary, dtype: object

In [25]:
mycf.salary[0]

'$9,000to$13,500'

In [ ]:
mycf.salary.map(lambda x: str(x).split('to').replace('$',''))

In [41]:
mycf.salary[0].split('to')

['$9,000', '$13,500']

In [ ]:
mycf.salary[0]

In [1]:
# driver.close()

NameError: name 'driver' is not defined